In [1]:
import pickle
import numpy as np
from load_pickle import get_outputs
from itertools import product
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import io
from scipy import stats
import pandas as pd
import seaborn as sns
import os
import sys

cwd = os.getcwd()
sys.path.append(cwd+"/../handy_functions")

from scipy.stats import mannwhitneyu, normaltest, circvar
from statannotations.Annotator import Annotator
from scipy.stats import circvar
from utils import *
from pull_pickles import make_DF, print_df

data_folder = '/Users/kendranoneman/Projects/mayo/data/neural-decoding/outpickles/'
fig_folder='/Users/kendranoneman/Projects/mayo/NeuralDecoding/figures/' 

#colors2 = ['purple', 'blue','cyan','mediumaquamarine','green','yellowgreen','gold','orange', 'magenta', 'red','gray'] #Colors to plot each method
colors = ['mediumorchid','seagreen','cornflowerblue','gold']

sessions = [18,20,22,24,28,29,32,33,34,38]

In [ ]:
make_DF(sessions)

DF = pd.read_csv(cwd+'/out.csv')
DF.head(5)

yay
10
20
30
40
50
60
70
80
90
100
110
120
130
140
yay
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
yay
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
yay
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
yay
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
yay
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
yay
10
20
30
40
50
60
70
80


In [ ]:
units = pd.DataFrame()
for s in sessions:
    u = pd.read_csv(cwd+'/../datasets/units-pa{:0>2d}dir4A-pre500-post300.csv'.format(s))
    units = pd.concat([units,u],ignore_index=True)
    
units.head(5)

## SNR 

In [ ]:
# SNR DISTRIBUTIONS
palette = {'MT': colors[0],'FEF': colors[1]}
a = sns.displot(units, x="SNR", hue="BrainArea", col="Session", binwidth=0.25, palette=palette, height=4, col_wrap=5,hue_order=["MT","FEF"])
def specs(x,hue, **kwargs):
    plt.axvline(x[hue=='MT'].mean(), c=colors[0], ls='--', lw=2)
    plt.axvline(x[hue=='FEF'].mean(), c=colors[1], ls='--', lw=2)
    plt.text(4, 20, 'n: {:d}'.format(x[hue=='MT'].size),color=colors[0])
    plt.text(4, 19, 'mean: {:.2f}'.format(x[hue=='MT'].mean()),color=colors[0])
    plt.text(4, 18, 'std: {:.2f}'.format(x[hue=='MT'].std()),color=colors[0])
    plt.text(4, 16, 'n: {:d}'.format(x[hue=='FEF'].size),color=colors[1])
    plt.text(4, 15, 'mean: {:.2f}'.format(x[hue=='FEF'].mean()),color=colors[1])
    plt.text(4, 14, 'std: {:.2f}'.format(x[hue=='FEF'].std()),color=colors[1])
    _, p = mannwhitneyu(x[hue=='MT'],x[hue=='FEF'],method="auto")
    plt.text(4, 12, 'p = {:.3f}'.format(p),color='black')

a.map(specs,'SNR','BrainArea')
a.set(yticks=np.arange(0,25,5))
a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)

a.fig.text(0.47, 0.01, 'SNR', ha='center', fontsize=14)
a.fig.text(0.001, 0.5, 'Number of Units', va='center', rotation='vertical', fontsize=14)

a.savefig(fig_folder+'snr_dist.png', dpi=300, bbox_inches='tight')

In [ ]:
############ SNR v. R2: individual sessions ################

from scipy import stats

df = DF.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_var','pd_var','mfr_mn','mfr_var','vfr_mn','vfr_var','dm_mn','dm_var','di_mn','di_var','si_mn','si_var']].mean().reset_index()
df = df.loc[(df['num_neurons'] <= 22) & (df['condition'] != 'mt+fef')]
df['R2_zscore'] = df.groupby(['condition','sess']).R2.transform(lambda x : stats.zscore(x,ddof=1)).values
df['snr_mn_zscore'] = df.groupby(['condition','sess']).snr_mn.transform(lambda x : stats.zscore(x,ddof=1)).values
df.sort_values(by='condition', ascending=False)
palette = {'mt only': colors[0],'fef only': colors[1],'mt=fef': colors[2]}
hue_order = [ 'fef only', 'mt only','mt=fef']

a = sns.relplot(data=df, x="snr_mn", y="R2", hue="condition",hue_order=hue_order,palette=palette,col='sess',col_wrap=5,size=df['num_neurons'],facet_kws={'sharey': False, 'sharex': False})
def specs(x,y,hue,**kwargs):
    ax = plt.gca()
    p = stats.pearsonr(x[hue=='mt only'].values, y[hue=='mt only'].values)
    ax.text(0.8, 0.9, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[0], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='fef only'].values, y[hue=='fef only'].values)
    ax.text(0.8, 0.85, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[1], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='mt=fef'].values, y[hue=='mt=fef'].values)
    ax.text(0.8, 0.8, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[2], transform = ax.transAxes)

a.map(specs,'snr_mn','R2','condition')

a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)

a.fig.text(0.47, 0.01, 'SNR (mean across neurons)', ha='center', fontsize=16)
a.fig.text(0.001, 0.5, '$R^2$', va='center', rotation='vertical', fontsize=16)

a.savefig(fig_folder+'snrVr2.png', dpi=300, bbox_inches='tight')

In [ ]:
############ SNR v. R2: all sessions ################
from scipy import stats

df = DF.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_var','pd_var','mfr_mn','mfr_var','vfr_mn','vfr_var','dm_mn','dm_var','di_mn','di_var','si_mn','si_var']].mean().reset_index()
df = df.loc[(df['num_neurons'] <= 22) & (df['condition'] != 'mt+fef')]
df['R2_zscore'] = df.groupby(['condition']).R2.transform(lambda x : stats.zscore(x,ddof=1)).values
df['snr_mn_zscore'] = df.groupby(['condition']).snr_mn.transform(lambda x : stats.zscore(x,ddof=1)).values
df.sort_values(by='condition', ascending=False)
palette = {'mt only': colors[0],'fef only': colors[1],'mt=fef': colors[2]}

a = sns.relplot(data=df, x="snr_mn", y="R2", hue="condition",palette=palette,size=df['num_neurons'])
def specs(x,y,hue,**kwargs):
    ax = plt.gca()
    p = stats.pearsonr(x[hue=='mt only'].values, y[hue=='mt only'].values)
    ax.text(0.8, 0.9, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[0], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='fef only'].values, y[hue=='fef only'].values)
    ax.text(0.8, 0.85, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[1], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='mt=fef'].values, y[hue=='mt=fef'].values)
    ax.text(0.8, 0.8, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[2], transform = ax.transAxes)


a.map(specs,'snr_mn','R2','condition')
a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)

a.fig.text(0.47, 0.01, 'SNR (mean across neurons)', ha='center', fontsize=14)
a.fig.text(0.001, 0.5, 'Prediction Accuracy ($R^2$)', va='center', rotation='vertical', fontsize=14)

a.savefig(fig_folder+'snrVr2_allSess.png', dpi=300, bbox_inches='tight')

In [ ]:
############ SVM (SNR v. R2): individual sessions #################

from plotting_fxns import svm_cloud
fig, axes = plt.subplots(2,int(len(sessions)/2),figsize=(25,12)) #Create figure (11 rows by 3 columns)
fig.tight_layout(w_pad=2)
for s in range(len(sessions)):
    df = DF.loc[DF['sess'] == sessions[s]]
    df = df.loc[(df['num_neurons'] <= np.array([df['nMT'].max(), df['nFEF'].max()]).min()) & (df['condition'] != 'mt+fef')]
    df = df.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_var','pd_var','mfr_mn','mfr_var','vfr_mn','vfr_var','dm_mn','dm_var','di_mn','di_var','si_mn','si_var']].mean().reset_index()
    
    df['R2_zscore'] = df.groupby(['condition']).R2.transform(lambda x : stats.zscore(x,ddof=1)).values
    df['snr_mn_zscore'] = df.groupby(['condition']).snr_mn.transform(lambda x : stats.zscore(x,ddof=1)).values
    conditions = [
        (df['condition'] == 'mt only'),
        (df['condition'] == 'fef only'),
        (df['condition'] == 'mt=fef')
        ]
    values = [int(float(0)), int(float(1)), int(float(2))]
    df['condition2'] = np.select(conditions, values)
    df = df.sample(frac=1).reset_index() 
    
    features = df[['snr_mn','R2']]
    label = df[['condition2']]
    colors = ['mediumorchid','seagreen','cornflowerblue','gold']
    if s<5:
        r=0
        c=s
    else:
        r=1
        c=s-5
       
    poly_accuracy = svm_cloud(features,label,axes[r,c],colors)
    axes[r,c].set_xlim(np.min(np.array(x_test)-0.1, axis=0)[0],np.max(np.array(x_test), axis=0)[0]+0.1)
    axes[r,c].set_ylim(np.min(np.array(x_test), axis=0)[1]-0.01,np.max(np.array(x_test), axis=0)[1]+0.01)
    axes[r,c].set_title('{} (accuracy:{:.2f}%)'.format(sessions[s],poly_accuracy*100))
    
fig.subplots_adjust(left=0.01)
fig.subplots_adjust(bottom=0.01)
fig.text(0.5, 0.01, '# of neurons', ha='center', fontsize=14)
fig.text(0.01, 0.5, 'Prediction Accuracy ($R^2$)', va='center', rotation='vertical', fontsize=14)
    
fig.savefig(fig_folder+'svm_snrVr2.png', dpi=300, bbox_inches='tight')

## Preferred direction

In [ ]:
# SNR DISTRIBUTIONS
palette = {'MT': colors[0],'FEF': colors[1]}
a = sns.displot(units, x="PrefDirFit", hue="BrainArea", col="Session", binwidth=45, palette=palette, height=4, col_wrap=5,hue_order=["MT","FEF"])
def specs(x,hue, **kwargs):
    plt.axvline(x[hue=='MT'].mean(), c=colors[0], ls='--', lw=2)
    plt.axvline(x[hue=='FEF'].mean(), c=colors[1], ls='--', lw=2)
    plt.text(4, 20, 'n: {:d}'.format(x[hue=='MT'].size),color=colors[0])
    plt.text(4, 19, 'mean: {:.2f}'.format(x[hue=='MT'].mean()),color=colors[0])
    plt.text(4, 18, 'std: {:.2f}'.format(x[hue=='MT'].std()),color=colors[0])
    plt.text(4, 16, 'n: {:d}'.format(x[hue=='FEF'].size),color=colors[1])
    plt.text(4, 15, 'mean: {:.2f}'.format(x[hue=='FEF'].mean()),color=colors[1])
    plt.text(4, 14, 'std: {:.2f}'.format(x[hue=='FEF'].std()),color=colors[1])
    _, p = mannwhitneyu(x[hue=='MT'],x[hue=='FEF'],method="auto")
    plt.text(4, 12, 'p = {:.3f}'.format(p),color='black')

a.map(specs,'PrefDirFit','BrainArea')
a.set(yticks=np.arange(0,25,5))
a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)

a.fig.text(0.47, 0.01, 'Preferred Direction', ha='center', fontsize=14)
a.fig.text(0.001, 0.5, 'Number of Units', va='center', rotation='vertical', fontsize=14)

a.savefig(fig_folder+'prefDir_dist.png', dpi=300, bbox_inches='tight')

In [ ]:
############ varPrefDir v. R2: individual sessions ################

from scipy import stats

df = DF.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_var','pd_var','mfr_mn','mfr_var','vfr_mn','vfr_var','dm_mn','dm_var','di_mn','di_var','si_mn','si_var']].mean().reset_index()
df = df.loc[(df['num_neurons'] <= 22) & (df['condition'] != 'mt+fef')]
#df['R2_zscore'] = df.groupby(['condition','sess']).R2.transform(lambda x : stats.zscore(x,ddof=1)).values
#df['snr_mn_zscore'] = df.groupby(['condition','sess']).snr_mn.transform(lambda x : stats.zscore(x,ddof=1)).values
df.sort_values(by='condition', ascending=False)
palette = {'mt only': colors[0],'fef only': colors[1],'mt=fef': colors[2]}
hue_order = [ 'fef only', 'mt only','mt=fef']

a = sns.relplot(data=df, x="pd_var", y="R2", hue="condition",hue_order=hue_order,palette=palette,col='sess',col_wrap=5,size=df['num_neurons'],facet_kws={'sharey': False, 'sharex': True})
def specs(x,y,hue,**kwargs):
    ax = plt.gca()
    p = stats.pearsonr(x[hue=='mt only'].values, y[hue=='mt only'].values)
    ax.text(0.8, 0.9, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[0], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='fef only'].values, y[hue=='fef only'].values)
    ax.text(0.8, 0.85, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[1], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='mt=fef'].values, y[hue=='mt=fef'].values)
    ax.text(0.8, 0.8, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[2], transform = ax.transAxes)
    ax.set_xlim((0,70))
a.map(specs,'pd_var','R2','condition')

a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)


a.fig.text(0.47, 0.01, 'Preferred Direction (circular variance across neurons)', ha='center', fontsize=16)
a.fig.text(0.001, 0.5, '$R^2$', va='center', rotation='vertical', fontsize=16)

a.savefig(fig_folder+'pdvarVr2.png', dpi=300, bbox_inches='tight')

In [ ]:
############ varPrefDir v. R2: all sessions ################
from scipy import stats

df = DF.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_var','pd_var','mfr_mn','mfr_var','vfr_mn','vfr_var','dm_mn','dm_var','di_mn','di_var','si_mn','si_var']].mean().reset_index()
df = df.loc[(df['num_neurons'] <= 22) & (df['condition'] != 'mt+fef')]
df.sort_values(by='condition', ascending=False)
palette = {'mt only': colors[0],'fef only': colors[1],'mt=fef': colors[2]}

a = sns.relplot(data=df, x="pd_var", y="R2", hue="condition",palette=palette,size=df['num_neurons'])
def specs(x,y,hue,**kwargs):
    ax = plt.gca()
    p = stats.pearsonr(x[hue=='mt only'].values, y[hue=='mt only'].values)
    ax.text(0.8, 0.9, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[0], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='fef only'].values, y[hue=='fef only'].values)
    ax.text(0.8, 0.85, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[1], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='mt=fef'].values, y[hue=='mt=fef'].values)
    ax.text(0.8, 0.8, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[2], transform = ax.transAxes)
    ax.set_xlim((0,70))

a.map(specs,'pd_var','R2','condition')
a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)

a.fig.text(0.47, 0.01, 'Preferred Direction (circular variance across neurons)', ha='center', fontsize=14)
a.fig.text(0.001, 0.5, 'Prediction Accuracy ($R^2$)', va='center', rotation='vertical', fontsize=14)

a.savefig(fig_folder+'pdvarVr2_allSess.png', dpi=300, bbox_inches='tight')

In [ ]:
############ SVM (varPrefDir v. R2): individual sessions #################

from plotting_fxns import svm_cloud
fig, axes = plt.subplots(2,int(len(sessions)/2),figsize=(25,12)) #Create figure (11 rows by 3 columns)
fig.tight_layout(w_pad=2)
for s in range(len(sessions)):
    df = DF.loc[DF['sess'] == sessions[s]]
    df = df.loc[(df['num_neurons'] <= np.array([df['nMT'].max(), df['nFEF'].max()]).min()) & (df['condition'] != 'mt+fef')]
    df = df.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_var','pd_var','mfr_mn','mfr_var','vfr_mn','vfr_var','dm_mn','dm_var','di_mn','di_var','si_mn','si_var']].mean().reset_index()
    
    #df['R2_zscore'] = df.groupby(['condition']).R2.transform(lambda x : stats.zscore(x,ddof=1)).values
    #df['snr_mn_zscore'] = df.groupby(['condition']).snr_mn.transform(lambda x : stats.zscore(x,ddof=1)).values
    conditions = [
        (df['condition'] == 'mt only'),
        (df['condition'] == 'fef only'),
        (df['condition'] == 'mt=fef')
        ]
    values = [int(float(0)), int(float(1)), int(float(2))]
    df['condition2'] = np.select(conditions, values)
    df = df.sample(frac=1).reset_index() 
    
    features = df[['pd_var','R2']]
    label = df[['condition2']]
    colors = ['mediumorchid','seagreen','cornflowerblue','gold']
    if s<5:
        r=0
        c=s
    else:
        r=1
        c=s-5
       
    poly_accuracy = svm_cloud(features,label,axes[r,c],colors,str(sessions[s]))
    
fig.subplots_adjust(left=0.01)
fig.subplots_adjust(bottom=0.01)
fig.text(0.5, 0.01, '# of neurons', ha='center', fontsize=14)
fig.text(0.01, 0.5, 'Preferred Direction (circular variance across neurons)', va='center', rotation='vertical', fontsize=14)
    
fig.savefig(fig_folder+'svm_varpdVr2.png', dpi=300, bbox_inches='tight')